In [1]:
import os
import random

In [2]:
cd ..

/kaggle


In [3]:
cd ..


/


In [4]:
ls

'=24.4'                                     libx32@
 NGC-DL-CONTAINER-LICENSE                   media/
 bin@                                       mnt/
 boot/                                      opt/
 cuda-keyring_1.0-1_all.deb                 proc/
 dev/                                       root/
 entrypoint.sh*                             run/
 etc/                                       run_jupyter.sh*
 home/                                      sbin@
 install_packages.sh*                       srv/
 kaggle/                                    sys/
 lib@                                       tmp/
 lib32@                                     usr/
 lib64@                                     var/
 libnvinfer8_8.6.1.6-1+cuda12.0_amd64.deb


In [5]:
### read words ###
text_file = open('kaggle/input/dataset/words_250000_train.txt',"r")
full_dictionary = text_file.read().splitlines()


In [6]:
cd /kaggle/working

/kaggle/working


In [7]:
import numpy as np

random.seed(40)

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

random.seed(40)

# Sample data for demonstration
words = full_dictionary

def generate_training_data_3(words):
    X = []
    y = []
    for word in words:
      # print(word)
      for char in np.unique(list(word)):
          X.append(word.replace(char,"_"))  # Input with missing character
          y.append(char)
          try:
            X.append(word.replace(char,"_").replace(random.choice(list(word.replace(char,""))),"_"))  # Input with missing character
            y.append(char)                      # Target character (the missing one)
          except:
            pass
          # print(word.replace(char,"_"))
    return X, y


# Toy dataset
X_train_3, y_train_3 = generate_training_data_3(words)

words_3 = X_train_3
labels_3 = y_train_3

max_length_3 = max(len(word) for word in words_3)

# Create a dictionary for character to index mapping
char_to_index = {char: idx for idx, char in enumerate(set(''.join(words_3)))}
index_to_char = {idx: char for char, idx in char_to_index.items()}

# Convert words and labels to numerical format
X_padded_3 = pad_sequences([[char_to_index[char] for char in word] for word in words_3], maxlen=max_length_3, padding='post')
y_padded_3 = np.array([char_to_index[label] for label in labels_3])

X_train_3, X_temp_3, y_train_3, y_temp_3 = train_test_split(X_padded_3, y_padded_3, test_size=0.3, random_state=42)
X_val_3, X_test_3, y_val_3, y_test_3 = train_test_split(X_temp_3, y_temp_3, test_size=0.5, random_state=42)

# print("Padded data:")
# for i in range(len(words)):
#     print(f"{words[i]} -> {labels[i]}")

# print("\nPadded and numerical data:")
# print(X_padded_2)
# print(y_padded_2)


# BiLSTM with attention

## attention 

## LSTM 256

## embedding - 50

## no dropout

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Define a checkpoint callback
checkpoint_callback_6 = ModelCheckpoint('best_model_6.keras', 
                                      monitor='val_loss', 
                                      save_best_only=True, 
                                      mode='min', 
                                      verbose=1)

early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                       patience=3, 
                                       mode='min', 
                                       verbose=1)

In [9]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Layer
from tensorflow.keras import Input, Model
import keras
# Define parameters
vocab_size = len(char_to_index)  # Size of vocabulary (unique characters)
embedding_dim = 50  # Dimension of character embeddings
lstm_units = 256  # Number of units in LSTM layers
num_layers = 3  # Number of BiLSTM layers
dropout_rate = 0.1  # Dropout rate

# Custom Attention Layer
@keras.saving.register_keras_serializable()
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

# Define Sequential model
model_6 = Sequential()

# Add layers to the model
model_6.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length_3))

# Add BiLSTM layers with Dropout
for _ in range(num_layers):
    model_6.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model_6.add(Dropout(rate=dropout_rate))

# Add Attention Layer
model_6.add(AttentionLayer())

# Output layer
model_6.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model_6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_6.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ ?                      │   0 (unbuilt) │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train the model
model_6.fit( X_train_3, 
            y_train_3, 
            epochs=3, 
            batch_size=256, 
            validation_data=(X_val_3, y_val_3), 
            verbose=1,callbacks=[checkpoint_callback_6,early_stopping_callback])

Epoch 1/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.3581 - loss: 2.0191
Epoch 1: val_loss improved from inf to 1.45258, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 770s 83ms/step - accuracy: 0.3581 - loss: 2.0190 - val_accuracy: 0.5045 - val_loss: 1.4526
Epoch 2/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5202 - loss: 1.3977
Epoch 2: val_loss improved from 1.45258 to 1.33420, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 766s 83ms/step - accuracy: 0.5202 - loss: 1.3977 - val_accuracy: 0.5361 - val_loss: 1.3342
Epoch 3/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5572 - loss: 1.2683
Epoch 3: val_loss improved from 1.33420 to 1.29104, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 767s 83ms/step - accuracy: 0.5572 - loss: 1.2683 - val_accuracy: 0.5480 - val_loss: 1.2910


In [12]:
# Train the model
model_6.fit( X_train_3, 
            y_train_3, 
            epochs=7, 
            batch_size=256, 
            validation_data=(X_val_3, y_val_3), 
            verbose=1,callbacks=[checkpoint_callback_6,early_stopping_callback])

Epoch 1/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5783 - loss: 1.1963
Epoch 1: val_loss improved from 1.29104 to 1.26879, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 768s 83ms/step - accuracy: 0.5783 - loss: 1.1963 - val_accuracy: 0.5529 - val_loss: 1.2688
Epoch 2/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5929 - loss: 1.1486
Epoch 2: val_loss improved from 1.26879 to 1.25708, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 766s 83ms/step - accuracy: 0.5929 - loss: 1.1486 - val_accuracy: 0.5563 - val_loss: 1.2571
Epoch 3/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.6031 - loss: 1.1154
Epoch 3: val_loss improved from 1.25708 to 1.25475, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 768s 84ms/step - accuracy: 0.6031 - loss: 1.1154 - val_accuracy: 0.5583 - val_loss: 1.2548
Epoch 4/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.6112 - loss: 1.0897
Epoch 4: val_loss imp

In [13]:
# checkpoint_callback_total6 = ModelCheckpoint('best_model_6.keras', 
#                                       monitor='val_loss', 
#                                       save_best_only=True, 
#                                       mode='min', 
#                                       verbose=1)

In [14]:
# Train the model
model_6.fit(X_train_3, 
            y_train_3, 
            epochs=3,      
            batch_size=256,                     
            validation_data=(X_val_3, y_val_3), 
            verbose=1,
            callbacks=[checkpoint_callback_6,early_stopping_callback])

Epoch 1/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.6290 - loss: 1.0319
Epoch 1: val_loss did not improve from 1.25157
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 768s 84ms/step - accuracy: 0.6290 - loss: 1.0319 - val_accuracy: 0.5614 - val_loss: 1.2569
Epoch 2/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.6325 - loss: 1.0216
Epoch 2: val_loss did not improve from 1.25157
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 768s 83ms/step - accuracy: 0.6325 - loss: 1.0216 - val_accuracy: 0.5606 - val_loss: 1.2518
Epoch 3/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.6350 - loss: 1.0155
Epoch 3: val_loss improved from 1.25157 to 1.25037, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 768s 84ms/step - accuracy: 0.6350 - loss: 1.0155 - val_accuracy: 0.5605 - val_loss: 1.2504


# Complete Training data
# randome shuffling 

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Layer
from tensorflow.keras import Input, Model
import keras
# Define parameters
vocab_size = len(char_to_index)  # Size of vocabulary (unique characters)
embedding_dim = 50  # Dimension of character embeddings
lstm_units = 256  # Number of units in LSTM layers
num_layers = 3  # Number of BiLSTM layers
dropout_rate = 0.1  # Dropout rate

# Custom Attention Layer
@keras.saving.register_keras_serializable()
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

# Define Sequential model
model_complete_6 = Sequential()

# Add layers to the model
model_complete_6.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length_3))

# Add BiLSTM layers with Dropout
for _ in range(num_layers):
    model_complete_6.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model_complete_6.add(Dropout(rate=dropout_rate))

# Add Attention Layer
model_complete_6.add(AttentionLayer())

# Output layer
model_complete_6.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model_complete_6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_complete_6.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer_1               │ ?                      │   0 (unbuilt) │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
from sklearn.utils import shuffle

X_train_complete_3, y_train_complete_3 = shuffle( X_padded_3, y_padded_3 )

# Train the model
model_complete_6.fit( X_train_complete_3, 
            y_train_complete_3,
            epochs=8, 
            batch_size=256, 
#             validation_data=(X_val_3, y_val_3), 
            verbose=1,
#             callbacks=[checkpoint_callback_6,early_stopping_callback]
           )

Epoch 1/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1025s 78ms/step - accuracy: 0.3869 - loss: 1.9100
Epoch 2/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1024s 78ms/step - accuracy: 0.5363 - loss: 1.3367
Epoch 3/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1030s 78ms/step - accuracy: 0.5664 - loss: 1.2294
Epoch 4/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1035s 79ms/step - accuracy: 0.5820 - loss: 1.1727
Epoch 5/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1042s 79ms/step - accuracy: 0.5911 - loss: 1.1403
Epoch 6/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1042s 79ms/step - accuracy: 0.5981 - loss: 1.1165
Epoch 7/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1041s 79ms/step - accuracy: 0.6033 - loss: 1.0981
Epoch 8/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 1036s 79ms/step - accuracy: 0.6076 - loss: 1.0836


In [17]:
model_complete_6.save('best_model_6_complete.keras')

In [18]:
model_complete_6_recon = tf.keras.models.load_model('best_model_6_complete.keras')
from IPython.display import FileLink
FileLink(r'best_model_6_complete.keras')
print("here pliss work i am pissed ")

here pliss work i am pissed 


In [19]:
# model_complete_6_recon = tf.keras.models.load_model('best_model_6_complete.keras')

In [20]:
## load model 
# model_6_recon = tf.keras.models.load_model('best_model_6.keras')

In [21]:
# Evaluate on test set
loss, accuracy = model_complete_6_recon.evaluate(X_test_3, y_test_3, verbose=1)
print(f"validation set Accuracy: {accuracy*100:.2f}%")

15762/15762 ━━━━━━━━━━━━━━━━━━━━ 131s 8ms/step - accuracy: 0.6345 - loss: 1.0027
validation set Accuracy: 63.64%
